In [1]:
import torch
import torch.nn.functional as F
import torch.optim as optim

from dhg.data import Cora
from dhg import Graph
from dhg.models import GCN
from dhg.metrics import GraphVertexClassificationEvaluator as Evaluator
from dhg.random import set_seed

import time
from copy import deepcopy

In [2]:
def train(A, X, lbls, net, train_idx, optimizer, epoch):

    net.train()

    st = time.time()
    optimizer.zero_grad()
    outs = net(X, A)
    outs, lbls = outs[train_idx], lbls[train_idx]
    loss = F.cross_entropy(outs, lbls)
    loss.backward()
    optimizer.step()

    print(f"Epoch: {epoch}, Time: {time.time()-st:.5f}s, Loss: {loss.item():.5f}")

    return loss.item()

In [3]:
@torch.no_grad()
def infer(A, X, lbls, net, idx, test=False):

    net.eval()

    outs = net(X, A)
    outs, lbls = outs[idx], lbls[idx]

    if not test:
        res = evaluator.validate(lbls, outs)
    else:
        res = evaluator.test(lbls, outs)

    return res

In [4]:
if __name__ == "__main__":

    set_seed(2022)

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    ################################################################
    data = Cora()
    G = Graph(data["num_vertices"], data["edge_list"])
    X, lbl = data["features"], data["labels"]

    train_mask = data["train_mask"]
    val_mask   = data["val_mask"]
    test_mask  = data["test_mask"]

    net = GCN(data["dim_features"], 16, data["num_classes"])
    optimizer = optim.Adam(net.parameters(), lr=0.01, weight_decay=5e-4)

    evaluator = Evaluator(["accuracy", "f1_score", {"f1_score": {"average": "micro"}}])

    ################################################################
    G = G.to(device)
    X, lbl = X.to(device), lbl.to(device)
    net = net.to(device)

    ################################################################
    best_val = 0
    best_state = None
    best_epoch = 0
    for epoch in range(300):

        # train, 每轮optimizer的参数在优化
        train(G, X, lbl, net, train_mask, optimizer, epoch)

        # validation
        if epoch % 1 == 0:
            with torch.no_grad():
                val_res = infer(G, X, lbl, net, val_mask)
            if val_res > best_val:
                print(f"update best: {val_res:.5f}")
                best_val   = val_res
                best_state = deepcopy(net.state_dict())
                best_epoch = epoch

    print("\ntrain finished!")
    print(f"best val: {best_val:.5f}")

    # test
    print("test...")
    net.load_state_dict(best_state)
    res = infer(G, X, lbl, net, test_mask, test=True)
    print(f"final result: epoch: {best_epoch}")
    print(res)

Epoch: 0, Time: 0.32276s, Loss: 1.95010
update best: 0.31600
Epoch: 1, Time: 0.00105s, Loss: 1.94601
Epoch: 2, Time: 0.00096s, Loss: 1.94383
Epoch: 3, Time: 0.00102s, Loss: 1.93907
Epoch: 4, Time: 0.00104s, Loss: 1.93350
update best: 0.38000
Epoch: 5, Time: 0.00095s, Loss: 1.92899
Epoch: 6, Time: 0.00096s, Loss: 1.92461
Epoch: 7, Time: 0.00098s, Loss: 1.91764
Epoch: 8, Time: 0.00093s, Loss: 1.91163
Epoch: 9, Time: 0.00094s, Loss: 1.90064
Epoch: 10, Time: 0.00166s, Loss: 1.89617
Epoch: 11, Time: 0.00087s, Loss: 1.88721
Epoch: 12, Time: 0.00082s, Loss: 1.87626
Epoch: 13, Time: 0.00085s, Loss: 1.86941
Epoch: 14, Time: 0.00087s, Loss: 1.86202
Epoch: 15, Time: 0.00082s, Loss: 1.85207
Epoch: 16, Time: 0.00084s, Loss: 1.84285
Epoch: 17, Time: 0.00085s, Loss: 1.83392
Epoch: 18, Time: 0.00081s, Loss: 1.82120
Epoch: 19, Time: 0.00080s, Loss: 1.80663
Epoch: 20, Time: 0.00072s, Loss: 1.79340
Epoch: 21, Time: 0.00077s, Loss: 1.77829
Epoch: 22, Time: 0.00070s, Loss: 1.76851
update best: 0.38800
Epoc